In [16]:
# from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

In [31]:
# llm = OpenAI(model='gpt-3.5-turbo-instruct')

# make template -> formatting -> predict(from prompt or message)
chat = ChatOpenAI(
    temperature=0.1
)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}?"
)

prompt = template.format(
    country_a="Mexico", 
    country_b="Thailand"
)

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [14]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage


messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in Italian."
    ),
    AIMessage(
        content="Ciao, mi chiamo Paolo!"
    ),
    HumanMessage(
        content="What is the distance between Mexico and Thailand? Also, what is you name?"
    )
]

chat.predict_messages(messages)

AIMessage(content='La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come posso aiutarti oggi?')

In [22]:
template = ChatPromptTemplate.from_messages(
    messages=[
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}? Also, what is you name?")
    ]
)

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας, το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [25]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
    
p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [33]:
# make template -> formatting -> predict message

template = ChatPromptTemplate.from_messages(
    messages=[
        ("system", '''You are a list generating machine. 
         Everything you are asked will be answered with a comma separated list of max {max_items}. 
         Do NOT reply with anything else.'''),
        ("human", "{question}")
    ]
)

prompt = template.format_messages(
    max_items=10,
    question="What are the planets?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto']

In [34]:
chain = template | chat | CommaOutputParser()

chain.invoke(
    {
        "max_items": 5,
        "question": "What are the pokemons?"
    }
)

['Pikachu', 'Charizard', 'Bulbasaur', 'Squirtle', 'Jigglypuff']